# Abstract

Create tables of the pointing telemetry.

# Environment

In [1]:
import os

In [2]:
# Standard C-string engineering DB
os.environ['ENG_RESTFUL_URL'] = 'http://iwjwdmscemweb.stsci.edu/JWDMSEngFqAccB72/TlmMnemonicDataSrv.svc/'

In [29]:
from astropy.table import Table, join

In [3]:
from jwst.lib.engdb_tools import ENGDB_Service

# Library

In [18]:
def make_table(mnemonic, values):
    """Make a table from a column of data from a mnemonic"""
    times = [o.isot for o in values.obstime]
    t = Table([times, values.value], names=('obstime', mnemonic), meta={'name': mnemonic})
    return t

# Main

In [7]:
engdb = ENGDB_Service()

## Define when

In [13]:
START_TIME = '2019-06-03T16:00'
END_TIME = '2019-06-03T18:09'

## Retrieve the parameters

In [22]:
params = {
    'SA_ZATTEST1':  None,
    'SA_ZATTEST2':  None,
    'SA_ZATTEST3':  None,
    'SA_ZATTEST4':  None,
    'SA_ZRFGS2J11': None,
    'SA_ZRFGS2J21': None,
    'SA_ZRFGS2J31': None,
    'SA_ZRFGS2J12': None,
    'SA_ZRFGS2J22': None,
    'SA_ZRFGS2J32': None,
    'SA_ZRFGS2J13': None,
    'SA_ZRFGS2J23': None,
    'SA_ZRFGS2J33': None,
    'SA_ZADUCMDX':  None,
    'SA_ZADUCMDY':  None,
}

In [23]:
for param in params:
    try:
        params[param] = engdb.get_values(
            param, START_TIME, END_TIME,
            include_obstime=True, zip=False
        )
    except Exception as exception:
        raise ValueError(
            'Cannot retrive {} from engineering.'
            '\nFailure was {}'.format(
                param,
                exception
            )
        )

## Build the table

In [27]:
tables = {}
for mnemonic in params:
    tables[mnemonic] = make_table(mnemonic, params[mnemonic])

In [41]:
full_table = tables['SA_ZATTEST1']

In [42]:
for mnemonic in tables:
    if mnemonic != 'SA_ZATTEST1':
        full_table = join(full_table, tables[mnemonic], join_type='outer')

In [45]:
full_table.show_in_browser(jsviewer=True)

In [47]:
full_table.write('/tmp/full_table.csv', format='csv')

## Build a table with only times where ZATTEST exists

In [50]:
zattest_table = tables['SA_ZATTEST1']

In [51]:
for mnemonic in tables:
    if mnemonic != 'SA_ZATTEST1':
        zattest_table = join(zattest_table, tables[mnemonic], join_type='inner')

In [53]:
zattest_table.show_in_browser(jsviewer=True)

In [55]:
zattest_table.write('/tmp/zattest_table.csv', format='csv')